# Using Mars Clock

This notebook shows how some of the basic functionality of Mars Clock works and how it can be used. We used the information from this website - https://www.giss.nasa.gov/tools/mars24/help/notes.html.

In [42]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import units as u

In [43]:
# Define some constants - Use the string format process to limit the precision to the actual measured precisions!
mars_solar_day = 24.*u.hour + 39.*u.minute + 35.244*u.second
mars_sol = mars_solar_day # This is what's referred to as a sol
mars_tropical_year_in_sols = 668.5921*mars_solar_day
mars_sidereal_day = 24.*u.hour + 37.*u.min + 22.663*u.second

mars_tropical_year = 668.5921*mars_sol
mars_sidereal_year = 668.5991*mars_sol

In [44]:
# Converting to Julian Date

# Make pandas Timestamp
ts = pd.Timestamp('2021-09-02 16:04:0')
# Convert to Julian Date
jd = ts.to_julian_date()
